Faker is used to generate fake data for testing and development purposes. This isnt necessary for the given task, but it can be useful for creating a realistic and consistent dataset for testing purposes for data quality and reliability checks.

In [1]:
!pip install pandas openpyxl Faker

Create dummy data for poc 

In [2]:
import os
import pandas as pd
import numpy as np
from faker import Faker
from setuptools import setup, find_packages


# Initialize Faker
fake = Faker()

# Data generation functions
def generate_hospitals(num_hospitals):
    hospitals = []
    for i in range(num_hospitals):
        hospitals.append({
            'hospital_id': f'H{i+1}',
            'name': fake.company() + ' Hospital',
            'location': fake.address()
        })
    return pd.DataFrame(hospitals)

def generate_clinics(num_clinics, num_hospitals):
    clinics = []
    for i in range(num_clinics):
        clinics.append({
            'clinic_id': f'C{i+1}',
            'name': fake.company() + ' Clinic',
            'location': fake.address(),
            'hospital_id': f'H{np.random.randint(1, num_hospitals + 1)}'
        })
    return pd.DataFrame(clinics)

def generate_providers(num_providers, num_hospitals, num_clinics):
    providers = []
    for i in range(num_providers):
        providers.append({
            'provider_id': f'P{i+1}',
            'name': fake.name(),
            'specialization': np.random.choice(['General Practitioner', 'Cardiologist', 'Dermatologist', 'Neurologist', 'Pediatrician', 'Social Care Worker', 'Bank Staff']),
            'associated_with': np.random.choice(['Hospital', 'Provider Company', 'Social Care Agency']),
            'organization': fake.company(),
            'hospital_id': f'H{np.random.randint(1, num_hospitals + 1)}' if np.random.choice([True, False]) else None,
            'clinic_id': f'C{np.random.randint(1, num_clinics + 1)}' if np.random.choice([True, False]) else None
        })
    return pd.DataFrame(providers)

def generate_patients(num_patients, providers):
    patients = []
    provider_ids = providers['provider_id'].tolist()
    hospital_ids = providers['hospital_id'].tolist()
    for i in range(num_patients):
        assigned_provider = np.random.choice(provider_ids)
        assigned_hospital = providers.loc[providers['provider_id'] == assigned_provider, 'hospital_id'].values[0]
        patients.append({
            'patient_id': f'PAT{i+1}',
            'name': fake.name(),
            'date_of_birth': fake.date_of_birth(minimum_age=0, maximum_age=90),
            'address': fake.address(),
            'phone': fake.phone_number(),
            'email': fake.email(),
            'assigned_provider_id': assigned_provider,
            'assigned_hospital_id': assigned_hospital
        })
    return pd.DataFrame(patients)

def generate_appointments(num_appointments, providers, patients):
    appointments = []
    for i in range(num_appointments):
        appointment_date = fake.date_time_this_year()
        waiting = np.random.choice([True, False])
        appointments.append({
            'appointment_id': f'APPT{i+1}',
            'patient_id': f'PAT{np.random.randint(1, len(patients) + 1)}',
            'provider_id': f'P{np.random.randint(1, len(providers) + 1)}',
            'appointment_date': appointment_date,
            'reason': np.random.choice(['Checkup', 'Follow-up', 'Consultation', 'Emergency']),
            'waiting': waiting,
            'waiting_time': np.random.randint(0, 120) if waiting else 0
        })
    return pd.DataFrame(appointments)

def generate_medical_assets(num_assets, num_hospitals, num_clinics):
    assets = []
    for i in range(num_assets):
        assets.append({
            'asset_id': f'A{i+1}',
            'type': np.random.choice(['MRI Machine', 'X-Ray Machine', 'Ultrasound Machine', 'CT Scanner', 'Defibrillator']),
            'manufacturer': fake.company(),
            'model': fake.word() + str(np.random.randint(100, 999)),
            'installation_date': fake.date_this_decade(),
            'status': np.random.choice(['Operational', 'Under Maintenance', 'Out of Service']),
            'hospital_id': f'H{np.random.randint(1, num_hospitals + 1)}',
            'clinic_id': f'C{np.random.randint(1, num_clinics + 1)}'
        })
    return pd.DataFrame(assets)

def generate_workforce(num_employees, num_hospitals):
    workforce = []
    for i in range(num_employees):
        workforce.append({
            'employee_id': f'E{i+1}',
            'name': fake.name(),
            'position': np.random.choice(['Nurse', 'Doctor', 'Technician', 'Administrative Staff']),
            'hospital_id': f'H{np.random.randint(1, num_hospitals + 1)}',
            'salary': fake.random_number(digits=5, fix_len=True),
            'hire_date': fake.date_this_decade()
        })
    return pd.DataFrame(workforce)

def generate_case_allocations(num_cases, providers, patients):
    case_allocations = []
    for i in range(num_cases):
        case_allocations.append({
            'case_id': f'C{i+1}',
            'patient_id': f'PAT{np.random.randint(1, len(patients) + 1)}',
            'provider_id': f'P{np.random.randint(1, len(providers) + 1)}',
            'case_description': fake.text(max_nb_chars=200),
            'case_status': np.random.choice(['Open', 'Closed', 'Pending'])
        })
    return pd.DataFrame(case_allocations)

def generate_data():
    num_hospitals = 10
    num_clinics = 30
    num_providers = 100
    num_patients = 200
    num_appointments = 500
    num_assets = 50
    num_employees = 100
    num_cases = 200
    
    hospitals = generate_hospitals(num_hospitals)
    clinics = generate_clinics(num_clinics, num_hospitals)
    providers = generate_providers(num_providers, num_hospitals, num_clinics)
    patients = generate_patients(num_patients, providers)
    appointments = generate_appointments(num_appointments, providers, patients)
    medical_assets = generate_medical_assets(num_assets, num_hospitals, num_clinics)
    workforce = generate_workforce(num_employees, num_hospitals)
    case_allocations = generate_case_allocations(num_cases, providers, patients)

    # Create data directory if not exists
    if not os.path.exists('data'):
        os.makedirs('data')

    hospitals.to_csv('data/hospitals.csv', index=False)
    clinics.to_csv('data/clinics.csv', index=False)
    providers.to_csv('data/providers.csv', index=False)
    patients.to_csv('data/patients.csv', index=False)
    appointments.to_csv('data/appointments.csv', index=False)
    medical_assets.to_csv('data/medical_assets.csv', index=False)
    workforce.to_csv('data/workforce.csv', index=False)
    case_allocations.to_csv('data/case_allocations.csv', index=False)
    print("Data generated successfully!")

# Generate data
generate_data()

# Test case script
def test_read_data_files():
    DATA_FILES = [
        'data/hospitals.csv',
        'data/clinics.csv',
        'data/providers.csv',
        'data/patients.csv',
        'data/appointments.csv',
        'data/medical_assets.csv',
        'data/workforce.csv',
        'data/case_allocations.csv'
    ]

    for file in DATA_FILES:
        try:
            df = pd.read_csv(file, low_memory=False)
            print(f"Successfully read {file}")
        except UnicodeDecodeError:
            df = pd.read_csv(file, encoding='latin-1', low_memory=False)
            print(f"Successfully read {file} with latin-1 encoding")
        
        print(df.head(4))
        
        
        # Check and convert data types if needed
        for col in df.columns:
            if not pd.api.types.is_numeric_dtype(df[col]) and not pd.api.types.is_string_dtype(df[col]):
                df[col] = df[col].astype(str)
                print(f"Column {col} in {file} converted to string")

# Run tests
test_read_data_files()

Data generated successfully!
Successfully read data/hospitals.csv
  hospital_id                      name  \
0          H1  Navarro-Hoffman Hospital   
1          H2  Cherry and Sons Hospital   
2          H3      Hughes-Ward Hospital   
3          H4  Morrison-Hansen Hospital   

                                            location  
0  83613 Victoria Circles Suite 899\nNew Cynthiap...  
1            1744 Gilmore Union\nMariaside, CA 31212  
2              838 Walker Rapids\nMccoyton, SD 12554  
3         2123 Johnson Mount\nNorth Steven, CA 93615  
Successfully read data/clinics.csv
  clinic_id                    name  \
0        C1        Hall-Hall Clinic   
1        C2       Bell Group Clinic   
2        C3        Olson PLC Clinic   
3        C4  Miller and Sons Clinic   

                                            location hospital_id  
0  5060 Amber Mountains\nEast Brittanymouth, IL 2...          H6  
1           267 Mayo Course\nHernandezstad, KY 17298         H10  
2          

Data Doctor Function to read data files and perform data cleaning and validation.

In [4]:
import pandas as pd
import os
import glob
import re
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from difflib import SequenceMatcher
from collections import defaultdict

def clean_column_names(df):
    df.columns = [re.sub(r'\W+', '_', col).lower() for col in df.columns]
    return df

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def is_pii(column_name):
    pii_keywords = ["name", "dob", "date of birth", "age", "contact number"]
    for keyword in pii_keywords:
        if similar(column_name.lower(), keyword) > 0.8:
            return True
    return False

def read_all_structured_files(directory_path):
    all_files = glob.glob(os.path.join(directory_path, "*.csv")) + glob.glob(os.path.join(directory_path, "*.xlsx"))
    all_sheets = []
    
    for file_path in all_files:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
            df = clean_column_names(df)
            all_sheets.append((file_path, df))
        elif file_path.endswith('.xlsx'):
            xls = pd.ExcelFile(file_path)
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                df = clean_column_names(df)
                all_sheets.append((f"{file_path} - {sheet_name}", df))
    
    return all_sheets

def find_critical_elements(all_sheets):
    column_files_map = defaultdict(list)
    for file_path, df in all_sheets:
        for column in df.columns:
            column_files_map[column].append(file_path)
    
    critical_elements = {column: files for column, files in column_files_map.items() if len(files) > 1}
    return critical_elements

def configure_quality_check(csv_file_path, excel_file_path=None):
    all_sheets = read_all_structured_files(os.path.dirname(csv_file_path))
    critical_elements = find_critical_elements(all_sheets)
    
    df = pd.read_csv(csv_file_path)
    df = clean_column_names(df)
    column_names = df.columns.tolist()

    pii_flags = []
    critical_data_elements = []
    
    for column in column_names:
        if is_pii(column):
            similar_columns = [col for sheet in all_sheets for col in sheet[1].columns if similar(col.lower(), column.lower()) > 0.8]
            description = ', '.join(set(similar_columns))
            pii_flags.append(f"Yes, description: {description}")
        else:
            pii_flags.append("No")
        
        if column in critical_elements:
            critical_data_elements.append(f"Yes, files: {', '.join(critical_elements[column])}")
        else:
            critical_data_elements.append("No")
    
    data_quality_checks_df = pd.DataFrame({
        "column_names": column_names,
        "PII_Flag": pii_flags,
        "test_completeness": ["Not Assessed" for _ in column_names],  # Set default value to "Not Assessed"
        "test_uniqueness": ["" for _ in column_names],
        "test_timeliness": ["" for _ in column_names],
        "test_consistency": ["" for _ in column_names],
        "test_accuracy": ["" for _ in column_names],
        "test_validity": ["" for _ in column_names],
        "critical_data_element": critical_data_elements
    })

    if not excel_file_path:
        excel_file_path = os.path.join(os.getcwd(), 'data_quality_checks_template.xlsx')

    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        data_quality_checks_df.to_excel(writer, sheet_name='Data Quality Checks', index=False, startrow=1)

    # Load the workbook to add the description
    workbook = load_workbook(excel_file_path)
    sheet = workbook['Data Quality Checks']
    
    # Add description at the top
    description = (f"File Name: {os.path.basename(csv_file_path)}\n"
                   "Please provide 'Yes' or 'No' in the columns below for each data quality check.")
    sheet['A1'] = description
    sheet.merge_cells('A1:H1')
    sheet['A1'].alignment = Alignment(wrap_text=True, vertical='center')

    # Adjust column widths
    for col in range(1, sheet.max_column + 1):
        max_length = 0
        column = get_column_letter(col)
        for cell in sheet[column]:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column].width = adjusted_width

    workbook.save(excel_file_path)
    print(f"Excel file '{excel_file_path}' created successfully with instructions.")


data_file_path = 'data/appointments.csv'
template_file_path = 'data/data_quality_checks_template.xlsx'

# Configure quality check and create template
configure_quality_check(data_file_path, template_file_path)

Excel file 'data/data_quality_checks_template.xlsx' created successfully with instructions.


Check data quality checks in the created Excel file.

In [14]:
import pandas as pd
import os
import re
from difflib import SequenceMatcher

def clean_column_names(df):
    df.columns = [re.sub(r'\W+', '_', col).lower() for col in df.columns]
    return df

def read_data_quality_template(excel_file_path):
    df_template = pd.read_excel(excel_file_path, sheet_name='Data Quality Checks', skiprows=1)
    return df_template

def assess_completeness(df, column_name):
    total_rows = len(df)
    missing_values = df[column_name].isnull().sum()
    non_missing_values = total_rows - missing_values
    completeness_percentage = (non_missing_values / total_rows) * 100

    completeness_df = pd.DataFrame({
        'Column Name': [column_name],
        'Total Rows': [total_rows],
        'Missing Values': [missing_values],
        'Non-Missing Values': [non_missing_values],
        'Completeness (%)': [round(completeness_percentage, 2)]
    })

    return completeness_df

def evaluate_data_quality(data_file_path, template_file_path):
    df_template = read_data_quality_template(template_file_path)

    if data_file_path.endswith('.csv'):
        df_data = pd.read_csv(data_file_path)
    elif data_file_path.endswith('.xlsx'):
        df_data = pd.read_excel(data_file_path)
    else:
        raise ValueError("Unsupported file format. Please use .csv or .xlsx files.")

    completeness_results = pd.DataFrame(columns=['Column Name', 'Total Rows', 'Missing Values', 'Non-Missing Values', 'Completeness (%)'])

    if not df_data.empty:
        for index, row in df_template.iterrows():
            column_name = row['column_names']
            test_completeness = str(row['test_completeness']).strip().lower() if pd.notna(row['test_completeness']) else 'not assessed'
            if test_completeness == 'yes':
                if column_name in df_data.columns:
                    completeness_df = assess_completeness(df_data, column_name)
                    if not completeness_df.empty:
                        completeness_results = pd.concat([completeness_results, completeness_df], ignore_index=True)
                else:
                    print(f"Warning: Column '{column_name}' not found in data file.")
            else:
                not_assessed_df = pd.DataFrame({
                    'Column Name': [column_name],
                    'Total Rows': ['N/A'],
                    'Missing Values': ['N/A'],
                    'Non-Missing Values': ['N/A'],
                    'Completeness (%)': ['Not Assessed']
                })
                completeness_results = pd.concat([completeness_results, not_assessed_df], ignore_index=True)
    else:
        print("Warning: The data file is empty.")

    if completeness_results.empty:
        print("No completeness analysis results to display.")
    else:
        print("Completeness analysis results:")
        print(completeness_results)

    return completeness_results

# Example usage:
data_file_path = 'data/appointments.csv'
template_file_path = 'data/data_quality_checks_template.xlsx'

# Evaluate data quality based on the template
completeness_results = evaluate_data_quality(data_file_path, template_file_path)

# Display the completeness results DataFrame
# print("Completeness Results DataFrame:")
# print(completeness_results)
completeness_results

Completeness analysis results:
        Column Name Total Rows Missing Values Non-Missing Values  \
0    appointment_id        500              0                500   
1        patient_id        500              0                500   
2       provider_id        500              0                500   
3  appointment_date        500              0                500   
4            reason        500              0                500   
5           waiting        500              0                500   
6      waiting_time        500              0                500   

   Completeness (%)  
0             100.0  
1             100.0  
2             100.0  
3             100.0  
4             100.0  
5             100.0  
6             100.0  


/tmp/ipykernel_80697/2353177093.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  completeness_results = pd.concat([completeness_results, completeness_df], ignore_index=True)


,Column Name,Total Rows,Missing Values,Non-Missing Values,Completeness (%)
0,appointment_id,500,0,500,100.0
1,patient_id,500,0,500,100.0
2,provider_id,500,0,500,100.0
3,appointment_date,500,0,500,100.0
4,reason,500,0,500,100.0
5,waiting,500,0,500,100.0
6,waiting_time,500,0,500,100.0
